In [ ]:
# Метрика Mean Average Precision
Перед запуском убедитесь в наличии установленного python-модуля pycocotools==2.0
Набор данных доступен по адресу cocodataset.org

In [ ]:
import tensorflow as tf
from iva_applications.utils import TPURunner
from mean_average_precision import get_network_outputs
from iva_applications.mscoco17.config import CLASS_NAMES
from map_utils import calculate_map
from iva_applications.yolo3.postprocess import build_detection_graph

In [ ]:
YOLO3_ANCHORS = [
            (10, 13), (16, 30), (33, 23),
            (30, 61), (62, 45), (59, 119),
            (116, 90), (156, 198), (373, 326)
]
iou_threshold = 0.5
confidence_threshold = 0.5

In [ ]:
# Инициализация IVA TPU предварительно скомпилированной программой
program_path = '<network_name>.tpu'
tpu_runner = TPURunner(program_path)
input_tensor_names = tpu_runner.input_nodes
conv_outputs_names = tpu_runner.output_nodes

In [ ]:
# Путь до набора данных
images_path = 'coco2017/val2017'
instances_path = 'coco17/annotations/instances_val2017.json'
conv_outputs_tensors = {
    'output_node_1/output:0': tf.placeholder(shape=(1, 19, 19, 255), dtype=tf.float32),
    'output_node_2/output:0': tf.placeholder(shape=(1, 38, 38, 255), dtype=tf.float32),
    'output_node_3/output:0': tf.placeholder(shape=(1, 76, 76, 255), dtype=tf.float32),
}

In [ ]:
# Построение графа постобработки
detections = build_detection_graph(
    conv_outputs_tensors,
    (608, 608),
    class_names=CLASS_NAMES,
    anchors=YOLO3_ANCHORS,
    iou_threshold=iou_threshold,
    confidence_threshold=confidence_threshold)

predictions, _, images = get_network_outputs(
    images_path,
    instances_path,
    tpu_runner,
    input_tensor_names=input_tensor_names,
    conv_outputs_names=conv_outputs_names,
    postprocessing_graph=detections,
    yolo_size=608)

In [ ]:
# Запуск вычисления метрики
mean_ap = calculate_map(instances_path, predictions, images)